In [ ]:
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import astropy as ap
import george
from george import kernels

In [ ]:
Vizier = Vizier(row_limit=20000)

In [ ]:
catalog = Vizier.get_catalogs("J/A+A/618/A93")

clucata = catalog[1]

In [ ]:
newc = clucata.group_by('Cluster')

maxcluster = np.argmax(newc.groups.indices[1:]-newc.groups.indices[:-1])
bigcluster = newc.groups[maxcluster+3]
print(bigcluster.colnames)
newc.groups[maxcluster+15]['Cluster'][0]

ASCC_99

Alessi_12

In [ ]:
cutcluster = bigcluster[bigcluster['PMemb']>.6]
cutcluster = cutcluster[~np.isnan(cutcluster["BP-RP"])]
plt.plot(cutcluster['RA_ICRS'],cutcluster['DE_ICRS'],'+')

In [ ]:
plt.plot(cutcluster['BP-RP'],cutcluster['Gmag'], '+')
plt.ylim(19, 7)

In [ ]:
plt.hist(cutcluster['PMemb'])

In [ ]:
kernel = kernels.Matern32Kernel(2)
gp = george.GP(kernel)

np.all(np.isreal(np.array(cutcluster["BP-RP"])))

In [ ]:
x = cutcluster["BP-RP"]
y = cutcluster["Gmag"]

In [ ]:
yerr = np.ones_like(cutcluster["Gmag"])*.25

In [ ]:
gp.compute(np.array(cutcluster["BP-RP"]), yerr)

In [ ]:
x_pred = np.linspace(0,3,1000)

In [ ]:
pred, pred_var = gp.predict(cutcluster['Gmag'], x_pred, return_var=True)

In [ ]:

plt.fill_between(x_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),
                color="k", alpha=0.2)
plt.plot(x_pred, pred, "k", lw=1.5, alpha=0.5)
plt.errorbar(cutcluster['BP-RP'], cutcluster['Gmag'], yerr=yerr, fmt=".k", capsize=0)
plt.ylim(18,7)

In [ ]:
from scipy.optimize import minimize

def neg_ln_like(p):
    gp.set_parameter_vector(p)
    return -gp.log_likelihood(y)

def grad_neg_ln_like(p):
    gp.set_parameter_vector(p)
    return -gp.grad_log_likelihood(y)

result = minimize(neg_ln_like, gp.get_parameter_vector(), jac=grad_neg_ln_like)
print(result)

gp.set_parameter_vector(result.x)
print("\nFinal ln-likelihood: {0:.2f}".format(gp.log_likelihood(y)))

In [ ]:
pred, pred_var = gp.predict(y, x_pred, return_var=True)

plt.fill_between(x_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),
                color="k", alpha=0.2)
plt.plot(x_pred, pred, "k", lw=1.5, alpha=0.5)
plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
plt.xlabel("GP-RP")
plt.ylabel("GMag")
plt.ylim(18,7.5)
plt.xlim(0,4.)